In [1]:
import json
import os
from datetime import datetime
import json
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix

import src.data.Dataset as dt

In [2]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

PYTHONPATH: :/Users/carlos/WORKSPACE/MegaClassifier/data/external/ai4eutils:/Users/carlos/WORKSPACE/MegaClassifier/data/external/CameraTraps:/Users/carlos/WORKSPACE/MegaClassifier/data/external/yolov5


In [3]:
initial_threshold = float(0.02)
steps_threshold = float(0.01)

positives_coberture = 0.0

OUTPUT_PATH = os.path.abspath("./resources/json")

MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5a.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5b.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v4.1.0.pb")

OUTPUT_FILE_PATH = os.path.join(OUTPUT_PATH, "detector_results.json")
TRAIN_CSV = os.path.abspath("./data/interim/train/20Train.csv")
REPORT_CSV = os.path.abspath("./reports/model_coberture")

os.makedirs(OUTPUT_PATH, exist_ok=True)

print(f"MODEL_PATH: {MODEL_PATH}")
print(f"OUTPUT_FILE_PATH: {OUTPUT_FILE_PATH}")
print(f"OUTPUT_PATH: {OUTPUT_PATH}")
print(f"TRAIN_CSV: {TRAIN_CSV}")

MODEL_PATH: /Users/carlos/WORKSPACE/MegaClassifier/models/MegaDetector/md_v5a.0.0.pt
OUTPUT_FILE_PATH: /Users/carlos/WORKSPACE/MegaClassifier/resources/json/detector_results.json
OUTPUT_PATH: /Users/carlos/WORKSPACE/MegaClassifier/resources/json
TRAIN_CSV: /Users/carlos/WORKSPACE/MegaClassifier/data/interim/train/20Train.csv


In [4]:
threshold = round(initial_threshold + steps_threshold, 2)
while positives_coberture != 1.0:
    threshold = round(threshold - steps_threshold, 2)

    detection_inicialitation_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

    !python data/external/CameraTraps/detection/run_detector_batch.py \
        "$MODEL_PATH" \
        /Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP \
        "$OUTPUT_FILE_PATH" \
        --recursive \
        --threshold "$threshold"

    with open(OUTPUT_FILE_PATH, "r") as file:
        data = json.load(file)

    for image in data["images"]:
        image["file"] = (
            image["file"]
            .replace(
                "/Users/carlos/WORKSPACE/MegaClassifier/dataset/emptyNonEmptyDatasetCROP",
                "",
            )
            .replace("/", "\\")
        )

    info = {
        "detection_initial_time": detection_inicialitation_time,
        "detection_completion_time": data["info"]["detection_completion_time"],
        "format_version": data["info"]["format_version"],
        "detector": data["info"]["detector"],
        "detector_threshold": threshold,
        "detector_metadata": data["info"]["detector_metadata"],
    }

    final_output = {
        "images": data["images"],
        "detection_categories": data["detection_categories"],
        "info": info,
    }

    threshold_str = str(threshold).replace(".", "_")
    json_name = f"{len(data['images'])}_images_{threshold_str}_threshold.json"
    model_name = os.path.basename(MODEL_PATH).split(".")[0]
    tmp_path = os.path.join(OUTPUT_PATH, model_name)
    os.makedirs(tmp_path, exist_ok=True)
    NEW_OUTPUT_FILE_PATH = os.path.join(tmp_path, json_name)

    with open(NEW_OUTPUT_FILE_PATH, "w") as f:
        json.dump(final_output, f, indent=1)

    os.remove(OUTPUT_FILE_PATH)

    data = final_output

    dataset = dt.load_from_csv(TRAIN_CSV)

    report_columns = [
        "file_name",
        "label",
        "threshold",
        "detector_label",
        "false_positive",
        "false_negative",
        "time_inference",
    ]
    report = pd.DataFrame(columns=report_columns)

    model = data["info"]["detector"].split(".")[0]

    for image in data["images"]:
        image_file = image["file"]
        indexes = dataset[dataset["file_name"] == image_file]

        if len(indexes) == 1:
            label = int(indexes["label"].iloc[0])
            detector_label = 1 if image["max_detection_conf"] > 0.0 else 0
            false_positive = int(label == 0 and detector_label == 1)
            false_negative = int(label == 1 and detector_label == 0)
            time_inference = image["time_inference"]

            new_row = {
                "file_name": image_file,
                "label": label,
                "threshold": data["info"]["detector_threshold"],
                "detector_label": detector_label,
                "false_positive": false_positive,
                "false_negative": false_negative,
                "time_inference": time_inference,
            }
            report = pd.concat([report, pd.DataFrame([new_row])], ignore_index=True)

    report_name = f"{len(data['images'])}_images_{model}.csv"
    REPORT_CSV_FILE = os.path.join(REPORT_CSV, report_name)

    if os.path.exists(REPORT_CSV_FILE):
        existing_report = dt.load_from_csv(REPORT_CSV_FILE)
        new_report = pd.concat([existing_report, report], ignore_index=True)        
        dt.dataset_to_csv(new_report, REPORT_CSV_FILE)
    else:
        dt.dataset_to_csv(report, REPORT_CSV_FILE)

    positivos_reales = report["label"].sum()
    positivos_cubiertos = report[
        (report["label"] == 1) & (report["detector_label"] == 1)
    ].shape[0]

    positives_coberture = (
        (positivos_cubiertos / positivos_reales) if positivos_reales > 0 else 0
    )
    
    print()
    print("-----------------------------------------------------------------------")
    print(f"El porcentaje de positivos cubiertos es {positives_coberture*100:.2f}%")
    print("-----------------------------------------------------------------------")
    print()


60 image files found in the input directory
PyTorch reports 0 available CUDA devices
GPU available: False
Using PyTorch version 1.10.1
Fusing layers... 
Model summary: 574 layers, 139990096 parameters, 0 gradients
Loaded model in 1.54 seconds
Loaded model in 1.54 seconds
100%|███████████████████████████████████████████| 60/60 [02:33<00:00,  2.55s/it]
Finished inference for 60 images in 2 minutes and 35.16 seconds
Output file saved at /Users/carlos/WORKSPACE/MegaClassifier/resources/json/detector_results.json
Done!
The file /Users/carlos/WORKSPACE/MegaClassifier/data/interim/train/20Train.csv has been successfully opened.
The dataset has been successfully saved to /Users/carlos/WORKSPACE/MegaClassifier/reports/model_coberture/60_images_md_v5a.csv

-----------------------------------------------------------------------
El porcentaje de positivos cubiertos es 96.43%
-----------------------------------------------------------------------

60 image files found in the input directory
PyTorch